# Homework 4

In [21]:
import networkx as nx

import random

## Visualize data

We chose the Google web graph at https://snap.stanford.edu/data/web-Google.html.

In [24]:
folder = 'data'
file = 'web-NotreDame.txt'

def read_graph(folder, file):
    with open(folder + '/' + file, 'r') as f:
        graph = nx.DiGraph()
        # skip first 4 lines
        for i in range(4):
            f.readline()
        # each line is an edge (pair of nodes)
        # counter = 0
        for line in f:
            # if counter == 1000:
            #     break
            edge = line.split()
            graph.add_edge(int(edge[0]), int(edge[1]))
            # counter += 1
    return graph

In [25]:
G = read_graph(folder, file)

print("Number of nodes: ", G.number_of_nodes())
print("Number of edges: ", G.number_of_edges())

Number of nodes:  325729
Number of edges:  1497134


In [ ]:
# visualize the graph
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(10, 10))
# nx.draw(G, node_size=10)
# plt.show()

In [26]:
# convert G to undirected graph
G_undirected = G.to_undirected()

print("Number of nodes: ", G_undirected.number_of_nodes())
print("Number of edges: ", G_undirected.number_of_edges())

Number of nodes:  325729
Number of edges:  1117563


In [28]:
# calculate number of triangles
triangles = nx.triangles(G_undirected)

print("Number of triangles: ", sum(triangles.values()) / 3)

Number of triangles:  8910005.0


## Streaming graph algorithm

In [49]:
stream = []

edge_res = [] # list of edges (i, j)
wedge_res = [] # list of wedges (i, j, k)
isClosed = [] # boolean array
tot_wedges = 0 # this is the total number of wedges formed by edges in the current edge res

def clean():
    global edge_res, wedge_res, isClosed, tot_wedges

    edge_res = []
    wedge_res = []
    isClosed = []
    tot_wedges = 0

def closed_by(wedge, edge):
    if edge[0] == wedge[2] and edge[1] == wedge[0]:
        return True
    return False

def update_tot_wedges():
    global tot_wedges

    for i in range(len(edge_res)):
        for j in range(i + 1, len(edge_res)):
            intersection = set(edge_res[i]).intersection(set(edge_res[j]))
            if len(intersection) == 1 and (edge_res[i][1] == edge_res[j][0] or edge_res[i][0] == edge_res[j][1]):
                tot_wedges += 1

def calculate_N_t(edge_t):
    N_t_list = []

    for i in range(len(edge_res)):
        intersection = set(edge_t).intersection(set(edge_res[i]))
        if len(intersection) == 1 and (edge_res[i][1] == edge_t[0] or edge_res[i][0] == edge_t[1]):
            N_t_list.append(i)

    return N_t_list

def streaming_triangles(s_e, s_w):
    global edge_res, wedge_res, tot_wedges

    k_t = []
    T_t = []

    # initialize edge_res, wedge_res with size s_e, s_w
    for i in range(s_e):
        edge_res.append((0, 0))
    for i in range(s_w):
        wedge_res.append((0, 0, 0))
        isClosed.append(False)

    for edge_t, t in zip(stream, range(1, len(stream) + 1)):
        print("=====================================")
        print("taking edge", edge_t, "at time", t)

        tot_wedges = 0
        
        update(edge_t, s_e, s_w, t)
        print("edge_res", edge_res)
        print("wedge_res", wedge_res)
        print("isClosed", isClosed)
        # count the fraction of entries in isClosed set to true
        p = sum(isClosed) / len(isClosed) if len(isClosed) != 0 else 0
        print("p", p)
        k_t.append(3 * p)
        print("k_t", k_t[t-1])
        print("(p * (t * t))", (p * (t * t)))
        print("s_e", s_e)
        print("(s_e * (s_e - 1))", (s_e * (s_e - 1)))
        print("tot_wedges", tot_wedges)
        T_t.append(((p * (t * t)) / (s_e * (s_e - 1))) * tot_wedges)

    return T_t

def update(edge_t, s_e, s_w, t):
    global isClosed, tot_wedges, edge_res, wedge_res

    for i in range(s_w):
        if closed_by(wedge_res[i], edge_t):
            isClosed[i] = True

    edge_res_old = edge_res.copy()

    for i in range(s_e):
        # pick a random number from 0 to 1
        x = random.random()
        if x <= 1 / t:
            edge_res[i] = edge_t
        # if there were any updates of edge res
    if edge_res != edge_res_old:
        # update tot_wedges, the number of wedges formed by edge_res
        update_tot_wedges()
        N_t_list = calculate_N_t(edge_t)
        print("N_t_list", N_t_list)
        new_wedges = len(N_t_list)

        for i in range(s_w):
            x = random.random()
            ratio = new_wedges / tot_wedges if tot_wedges != 0 else 0
            if x <= ratio:
                wedge_res[i] = (edge_t[0], edge_res[i][0], edge_res[i][1])
                wedge_res[i] = (edge_res[i][0], edge_res[i][1], edge_t[1])
                # pick uniform random wedge in N_t_list
                w = random.randint(0, len(N_t_list) - 1)
                print("w", w)
                index = N_t_list[w]
                print("N_t_list[w]", N_t_list[w])
                if edge_t[1] == edge_res[index][0]:
                    wedge_res[i] = (edge_t[0], edge_t[1], edge_res[index][1])
                else:
                    wedge_res[i] = (edge_res[index][0], edge_t[0], edge_t[1])
                isClosed[i] = False

In [185]:
stream = [(1, 2), (1, 3), (4, 1), (2, 3), (2, 4), (3, 4)]

clean()

print(streaming_triangles(6, 6))

taking edge (1, 2) at time 1
N_t_list []
edge_res [(1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2)]
wedge_res [(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]
isClosed [False, False, False, False, False, False]
p 0.0
k_t 0.0
(p * (t * t)) 0.0
s_e 6
(s_e * (s_e - 1)) 30
tot_wedges 0
taking edge (1, 3) at time 2
N_t_list []
edge_res [(1, 3), (1, 2), (1, 3), (1, 3), (1, 3), (1, 2)]
wedge_res [(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]
isClosed [False, False, False, False, False, False]
p 0.0
k_t 0.0
(p * (t * t)) 0.0
s_e 6
(s_e * (s_e - 1)) 30
tot_wedges 0
taking edge (4, 1) at time 3
N_t_list [0, 5]
w 1
N_t_list[w] 5
w 1
N_t_list[w] 5
w 0
N_t_list[w] 0
edge_res [(1, 3), (4, 1), (4, 1), (4, 1), (4, 1), (1, 2)]
wedge_res [(0, 0, 0), (4, 1, 2), (4, 1, 2), (0, 0, 0), (0, 0, 0), (4, 1, 3)]
isClosed [False, False, False, False, False, False]
p 0.0
k_t 0.0
(p * (t * t)) 0.0
s_e 6
(s_e * (s_e - 1)) 30
tot_wedges 8
taking edge (2, 3) at time 4
N_t_list [5]


In [ ]:
# TODO: run on the real dataset and check if the triangles are correct because we know the original number of triangles
# TODO: understand the k_t and T_t formulas
# Better answers in the final consideration

## Observations

- Diminuishing n_w result to lower the chances to close a triangle. When n_w approaches 0 then we're not estimating at all, we're just outputting 0 as an estimate
- Reducing n_e lower the chances to create a new wedge, that in turn lowet the chances to close a triangle. When n_e approaches 0 then we're not adding any wedge at all
- The approx. number of triangles in the graph (https://math.stackexchange.com/questions/823481/number-of-triangles-in-a-graph-based-on-number-of-edges) is abs(E)^1/2, where E is the number of edges in the graph
- We can use this number to set the initial values of n_w and n_e
- A suggestion is to set n_w = abs(E)^1/2 and n_e = 2 * n_w

## Final questions

1. What were the challenges you faced when implementing the algorithm?
2. Can the algorithm be easily parallelized? If yes, how? If not, why? Explain. Yes, it can be parallelized because for both edge_res and wedge_res we are trying to save the current edge or wedge in each position of the two array. Therefore, if a overwriting occurs, due to the parallelization, we are safe since we are not losing any information
3. Does the algorithm work for unbounded graph streams? Explain. No
4. Does the algorithm support edge deletions? If not, what modification would it need? Explain. Yes